In [1]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
import lxml.html
# from pywebcopy import save_webpage

In [2]:

internal_urls = set()
external_urls = set()

In [3]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [4]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [5]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    links = get_all_urls(url)
    for link in links:
        if total_urls_visited > max_urls:
            break
        crawl(link, max_urls=max_urls)


In [11]:
base_urls = ["https://www.clemson.edu/coronavirus/"]
parent_folder = "Doctoral" #Doctoral, Masters, Bach

In [ ]:
for base_url in base_urls:
    parsedurl = urlparse(base_url)
    # base_url_text = base_url.split("//",1)[1]
    base_url_text = parsedurl.netloc+parsedurl.path
    print(base_url_text)
    # base_url_text_domain = base_url_text.split("/",1)[0]
    base_url_text_domain = parsedurl.netloc
    print(base_url_text_domain)
    crawl(base_url)
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    
    
    count = 0
    last_count = 989874
    immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

    for i in range(0,4):
        for url in internal_urls:
            if base_url_text in url:
                immediate_urls.add(url)
        count = len(immediate_urls)
        if(count == last_count):
            break
        last_count = count
        print(count,i)
        for immediate_url in immediate_urls:
            crawl(immediate_url)
            print("[+] NEW Total External links:", len(external_urls))
            print("[+] NEW Total Internal links:", len(internal_urls))
            print("[+] NEW Total:", len(external_urls) + len(internal_urls))
            if len(internal_urls)>1000:
                break
        
        
    def removeAfterN(yourStr, nth, occurenceOf):
        return occurenceOf.join(yourStr.split(occurenceOf)[:nth])

    semifinal_immediate_urls = set()
    for immediate_url in immediate_urls:
        stripped_url = removeAfterN(immediate_url,2,"http")
        semifinal_immediate_urls.add(stripped_url)
    #     final_immediate_urls.add(rest)
    #     print(rest)

    print(semifinal_immediate_urls)
    total_count = len(semifinal_immediate_urls)
    print(total_count)
    
    final_immediate_urls = set()
    for semifinal_immediate_url in semifinal_immediate_urls:
        if "tel:" not in semifinal_immediate_url:
            final_immediate_urls.add(semifinal_immediate_url)

    print(final_immediate_urls)
    total_count = len(final_immediate_urls)
    print(total_count)
    
    name_of_folder = f'{base_url_text_domain}_Screenshots'
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
    if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
        os.makedirs(f'{parent_folder}\\{name_of_folder}')
    
    webpage_save = set()
    webpage_save_count = 0
    webpage_cannot_save = 0
    for i in final_immediate_urls:
        if ' ' in i:
            continue
        try:
            code=urlopen(i).getcode()
            webpage_save.add(i)
            webpage_save_count = webpage_save_count+1
            print(f"{code} can_save - {i}")
        except urllib.error.HTTPError as e:
            if e.code not in (200,300):
                webpage_cannot_save = webpage_cannot_save + 1
                print(f'{e.code} cannot_save - {i}')
                continue

    print(f'Webpages save as HTML :{webpage_save_count}')
    print(f'Webpages unable to save as HTML :{webpage_cannot_save}')

    pdf_count = 0
    pdf_urls = set()
    for webpage in webpage_save:
        if webpage[-4:] == ".pdf":
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                file_name = "Home"
            pdf_urls.add(webpage)
            print(webpage)
            r = requests.get(url, stream=True)
            with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                for chunk in r.iter_content(2000):
                    fd.write(chunk)
            pdf_count = pdf_count + 1

    print(f"Pdfs saved: {pdf_count}")
    
    final_immediate_urls = final_immediate_urls - pdf_urls
    webpage_save = webpage_save - pdf_urls
    web_save_count = len(webpage_save)
    total_count = len(final_immediate_urls)
    print(total_count) 
    
    
    webpage_save_counter_loop = 0
    for webpage in webpage_save:
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
        webpage_save_counter_loop = webpage_save_counter_loop+1
        print(f'{webpage_save_counter_loop} of {webpage_save_count} - {webpage}')

    print("Saving Completed")
    
    
    count = 0
    for i in final_immediate_urls:
        url = i
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        count = count + 1
        print(f"{count} of {total_count}")
        print(f'Visiting {base_url_text}/{file_name}')
        print(f'...Taking a screenshot')
        driver.get(url)
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
        print(f'Screenshot of {file_name} page taken! \n')
    #     print("Saving HTML")
    #     pyautogui.hotkey('ctrl', 's')
    #     time.sleep(1)
    #     pyautogui.typewrite(f'{file_name}.html')
    #     pyautogui.hotkey('enter')
    
    driver.quit()
    print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

    print("------------------------END-------------------------------\n")

www.clemson.edu/coronavirus/
www.clemson.edu
[+] Total External links: 2188
[+] Total Internal links: 1956
[+] Total: 4144
1417 0
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal links: 1956
[+] NEW Total: 4144
[+] NEW Total External links: 2188
[+] NEW Total Internal li

[+] NEW Total External links: 2203
[+] NEW Total Internal links: 1973
[+] NEW Total: 4176
[+] NEW Total External links: 2203
[+] NEW Total Internal links: 1973
[+] NEW Total: 4176
[+] NEW Total External links: 2203
[+] NEW Total Internal links: 1973
[+] NEW Total: 4176
[+] NEW Total External links: 2203
[+] NEW Total Internal links: 1973
[+] NEW Total: 4176
[+] NEW Total External links: 2203
[+] NEW Total Internal links: 1973
[+] NEW Total: 4176
[+] NEW Total External links: 2203
[+] NEW Total Internal links: 1973
[+] NEW Total: 4176
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/research/index.html/research/development/research/development/research/development/research/development/funding-opportunities/funding/tel:18646563311/tel:18646563311/tel:18646563311/facilities.html
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/research/index.html/research/development/research/development/

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 2217
[+] NEW Total Internal links: 1983
[+] NEW Total: 4200
[+] NEW Total External links: 2217
[+] NEW Total Internal links: 1983
[+] NEW Total: 4200
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/https://www.clemson.edu/coronavirus/faculty-staff/index.html/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/research/index.html
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/https://www.clemson.edu/coronavirus/faculty-staff/index.html/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/documents/DeerOaksResources
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/https://www.clemson.edu/coronavirus/faculty-staff/index.html/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/ADA_guidance
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/https://www.clemson.edu/coro

[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[+] NEW Total External links: 2261
[+] NEW Total Internal links: 2008
[+] NEW Total: 4269
[INT] Inte

...

[+] NEW Total External links: 3331
[+] NEW Total Internal links: 2997
[+] NEW Total: 6328
[+] NEW Total External links: 3331
[+] NEW Total Internal links: 2997
[+] NEW Total: 6328
[+] NEW Total External links: 3331
[+] NEW Total Internal links: 2997
[+] NEW Total: 6328
[+] NEW Total External links: 3331
[+] NEW Total Internal links: 2997
[+] NEW Total: 6328
[+] NEW Total External links: 3331
[+] NEW Total Internal links: 2997
[+] NEW Total: 6328
[+] NEW Total External links: 3332
[+] NEW Total Internal links: 2997
[+] NEW Total: 6329
[+] NEW Total External links: 3332
[+] NEW Total Internal links: 2997
[+] NEW Total: 6329
[+] NEW Total External links: 3332
[+] NEW Total Internal links: 2997
[+] NEW Total: 6329
[+] NEW Total External links: 3333
[+] NEW Total Internal links: 2997
[+] NEW Total: 6330
[+] NEW Total External links: 3333
[+] NEW Total Internal links: 2997
[+] NEW Total: 6330
[+] NEW Total External links: 3333
[+] NEW Total Internal links: 2997
[+] NEW Total: 6330
[+] NEW To

[+] NEW Total External links: 3363
[+] NEW Total Internal links: 3019
[+] NEW Total: 6382
[+] NEW Total External links: 3363
[+] NEW Total Internal links: 3019
[+] NEW Total: 6382
[+] NEW Total External links: 3364
[+] NEW Total Internal links: 3019
[+] NEW Total: 6383
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/https://www.clemson.edu/coronavirus/research/index.html/tel:18646563311/research/development/funding-opportunities/funding/research/development/funding-opportunities/funding/tel:18646563311/facilities.html
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/https://www.clemson.edu/coronavirus/research/index.html/tel:18646563311/research/development/funding-opportunities/funding/research/development/funding-opportunities/research/development/funding-opportunities/funding/covid19.html
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/https://www.clemson.edu/coronavirus/research/index.html/tel:18646563311/research/development/fundin

[+] NEW Total External links: 3395
[+] NEW Total Internal links: 3042
[+] NEW Total: 6437
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/student/index.html/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/tel:18646563311/
[+] NEW Total External links: 3396
[+] NEW Total Internal links: 3043
[+] NEW Total: 6439
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/research/index.html/research/development/funding-opportunities/funding/research/development/research/development/funding-opportunities/funding/tel:18646563311/tel:18646563311/tel:18646563311/facilities.html
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/research/index.html/research/development/funding-opportunities/funding/research/development/research/development/fund

[+] NEW Total External links: 3415
[+] NEW Total Internal links: 3067
[+] NEW Total: 6482
[+] NEW Total External links: 3415
[+] NEW Total Internal links: 3067
[+] NEW Total: 6482
[+] NEW Total External links: 3415
[+] NEW Total Internal links: 3067
[+] NEW Total: 6482
[+] NEW Total External links: 3416
[+] NEW Total Internal links: 3067
[+] NEW Total: 6483
[+] NEW Total External links: 3417
[+] NEW Total Internal links: 3067
[+] NEW Total: 6484
[+] NEW Total External links: 3418
[+] NEW Total Internal links: 3067
[+] NEW Total: 6485
[+] NEW Total External links: 3418
[+] NEW Total Internal links: 3067
[+] NEW Total: 6485
[+] NEW Total External links: 3418
[+] NEW Total Internal links: 3067
[+] NEW Total: 6485
[+] NEW Total External links: 3418
[+] NEW Total Internal links: 3067
[+] NEW Total: 6485
[+] NEW Total External links: 3418
[+] NEW Total Internal links: 3067
[+] NEW Total: 6485
[+] NEW Total External links: 3419
[+] NEW Total Internal links: 3067
[+] NEW Total: 6486
[+] NEW To

[+] NEW Total External links: 3435
[+] NEW Total Internal links: 3087
[+] NEW Total: 6522
[+] NEW Total External links: 3436
[+] NEW Total Internal links: 3087
[+] NEW Total: 6523
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/faculty-staff/index.html/tel:18646563311/tel:18646563311/research/tel:18646563311/research/research/research/index.html
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/faculty-staff/index.html/tel:18646563311/tel:18646563311/research/tel:18646563311/research/research/documents/DeerOaksResources
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/faculty-staff/index.html/tel:18646563311/tel:18646563311/research/tel:18646563311/research/research/ADA_guidance
[INT] Internal link: http://owstools.www.clemson.edu/textify/app/http://www.clemson.edu/coronavirus/faculty-staff/index.html/tel:18646563311/tel:18646563311/re